<a href="https://colab.research.google.com/github/ParkEunHyeok/Project_Kermit/blob/main/Transformer/Kermit_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install konlpy

In [7]:
import os
import re
import json

import numpy as np
import pandas as pd
from tqdm import tqdm

from konlpy.tag import Okt

In [5]:
# 구글 드라이브 연결
import os
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/HelloNewWorld/"

Mounted at /content/gdrive/


In [8]:
FILTERS = "([~.,!?\"':;)(])"
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)

MAX_SEQUENCE = 25

In [13]:
def load_data(path):
  data_df = pd.read_csv(path, header=0)
  question, answer = list(data_df['Q']), list(data_df['A'])
  return question, answer

In [16]:
def data_tokenizer(data):
  words = []
  for sentence in data:
    # normalize를 위해 필요 없는 특수 기호를 제거하는 부분
    sentence = re.sub(CHANGE_FILTER, "", sentence)
    for word in sentence.split():
      words.append(word)
  # tokenize, normalize를 통해 만들어진 값 반환
  return [word for word in words if word]

In [17]:
def prepro_like_morphlized(data):
  morph_analyzer = Okt()
  result_data = list()
  for seq in tqdm(data):
    morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
    result_data.append(morphlized_seq)
  return result_data

In [27]:
def load_vocabulary(path, vocab_path, tokenize_as_morph=False):
  # vocab의 배열
  vocabulary_list = []

  if not os.path.exists(vocab_path):
    if os.path.exists(path):
      # 데이터가 존재하면 pandas로 load
      data_df = pd.read_csv(path, encoding='utf-8')
      quesiton, answer = list(data_df['Q']), list(data_df['A'])

      if tokenize_as_morph:
        # 형태소에 따른 tokenizer 처리
        question = prepro_like_morphlized(quesiton)
        answer = prepro_like_morphlized(answer)
      # 형태소 분석한 후 data 배열로 합쳐줌
      data = []
      data.extend(quesiton)
      data.extend(answer)
      
      words = data_tokenizer(data)  # 특수기호 필터
      words = list(set(words))  # 중복 제거

      # 데이터 없는 내용 중에 MARKER를 사전에 추가
      # 리스트의 첫번째 부터
      # 순서대로 넣기 위해서 인덱스 0에 추가함
      # PAD = "<PADDING>"
      # STD = "<START>"
      # END = "<END>"
      # UNK = "<UNKNWON>"
      words[:0] = MARKER
    # word를 파일로 저장
    with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
      for word in words:
        vocabulary_file.write(word+'\n')

  # 파일이 이미 있으면 불러와서 배열에 넣어줌.
  with open(vocab_path, 'r', encoding='utf-8') as vocabulary_file:
    for line in vocabulary_file:
      vocabulary_list.append(line.strip())

  # 배열에 내용을 딕셔너리 구조로 만듦
  char2idx, idx2char = make_vocabulary(vocabulary_list)

  return char2idx, idx2char, len(char2idx)

In [20]:
def make_vocabulary(vocabulary_list):
  # dict, key=word, value=index
  char2idx = {char: idx for idx, char in enumerate(vocabulary_list)}
  # dict, key=index, value=word
  idx2char = {idx: char for idx, char in enumerate(vocabulary_list)}
  
  return char2idx, idx2char

In [21]:
def enc_processing(value, dictionary, tokenize_as_morph=False):
  # index값들을 가지고 있는 배열
  sequences_input_index = []
  # 하나의 인코딩 되는 문장의 길이를 가짐
  sequences_length = []

  # 형태소 분석기 사용 유무
  if tokenize_as_morph:
    value = prepro_like_morphlized(value)
  
  # 한 줄씩 불러옴
  for sequence in value:
    # 특수기호 필터링
    sequence = re.sub(CHANGE_FILTER, "", sequence)
    # 하나의 문장을 인코딩 할 때 가지고 있기 위한 배열
    sequence_index = []

    # 문장을 스페이스 단위로 자르고
    for word in sequence.split():
      # word가 딕셔너리에 존재하는지 보고
      # 있으면 sequence_index에 추가
      if dictionary.get(word) is not None:
        sequence_index.extend([dictionary[word]])
      # 없으면 UNK
      else:
        sequence_index.extend([dictionary[UNK]])
    
    # 문장 제한 길이보다 길어질 경우 뒤에 강제로 토큰을 잘라냄
    if len(sequence_index) > MAX_SEQUENCE:
      sequence_index = sequence_index[:MAX_SEQUENCE]
    
    # 하나의 문장에 길이를 넣어줌
    sequences_length.append(len(sequence_index))
    # 0 padding
    sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
    # 인덱스한 값을 배열에 넣어줌
    sequences_input_index.append(sequence_index)

  # 인덱스화된 배열을 numpy 배열로 변환하여 반환
  return np.asarray(sequences_input_index), sequences_length

In [36]:
def dec_output_precessing(value, dictionary, tokenize_as_morph=False):
  # 인덱스 값들을 가진 배열
  sequences_output_index = []
  # 하나의 디코딩 입력 되는 문장의 길이
  sequences_length = []

  # 형태소 분석기 사용 유무
  if tokenize_as_morph:
    value = prepro_like_morphlized(value)
  for sequence in value:
    # 특수 문자 필터링
    sequence = re.sub(CHANGE_FILTER, "", sequence)
    # 하나의 문장을 인코딩 할 때 가지고 있기 위한 배열
    sequence_index = []

    # 디코딩 처음에 [START]토큰 부여,
    # word를 가져와서 dict의 index를 넣어줌
    sequence_index = [dictionary[STD]] + [dictionary[word] if word in dictionary else dictionary[UNK] for word in sequence.split()]

    # 문장 제한 길이보다 길어질 경우 뒤에 강제로 토큰을 잘라냄
    if len(sequence_index) > MAX_SEQUENCE:
      sequence_index = sequence_index[:MAX_SEQUENCE]

    # 하나의 문장에 길이를 넣어줌
    sequences_length.append(len(sequence_index))
    # 0 padding
    sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
    # 인덱스한 값을 배열에 넣어줌
    sequences_output_index.append(sequence_index)
  return np.asarray(sequences_output_index), sequences_length

In [23]:
def dec_target_processing(value, dictionary, tokenize_as_morph=False):
  sequences_target_index = []

  if tokenize_as_morph:
    value = prepro_like_morphlized(value)

  for sequence in value:
    sequence = re.sub(CHANGE_FILTER, "", sequence)

    # 디코딩 출력의 마지막에 END 넣어줌
    sequence_index = [dictionary[word] if word in dictionary else dictionary[UNK] for word in sequence.split()]

    # 최대 길이보다 길어지면 뒤에 토큰을 잘라줌
    if len(sequence_index) >= MAX_SEQUENCE:
      sequence_index = sequence_index[:MAX_SEQUENCE - 1] + [dictionary[END]]
    else:
      sequence_index += [dictionary[END]]
    
    # 0 padding
    sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[END]]
    sequences_target_index.append(sequence_index)

  return np.asarray(sequences_target_index)

In [48]:
PATH = path+'data_in/ChatBotData.csv'
VOCAB_PATH = path+'data_in/vocabulary.txt'

In [49]:
inputs, outputs = load_data(PATH)

In [50]:
char2idx, idx2char, vocab_size = load_vocabulary(PATH, VOCAB_PATH, tokenize_as_morph=False)

In [51]:
index_inputs, input_seq_len = enc_processing(inputs, char2idx, tokenize_as_morph=False)
index_outputs, output_seq_len = dec_output_precessing(outputs, char2idx, tokenize_as_morph=False)
index_targets = dec_target_processing(outputs, char2idx, tokenize_as_morph=False)

In [52]:
data_configs = {}
data_configs['char2idx'] = char2idx
data_configs['idx2char'] = idx2char
data_configs['vocab_size'] = vocab_size
data_configs['pad_symbol'] = PAD
data_configs['std_symbol'] = STD
data_configs['end_symbol'] = END
data_configs['unk_symbol'] = UNK

In [53]:
DATA_IN_PATH = path+'data_in/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

np.save(open(DATA_IN_PATH + TRAIN_INPUTS, 'wb'), index_inputs)
np.save(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'wb'), index_outputs)
np.save(open(DATA_IN_PATH + TRAIN_TARGETS , 'wb'), index_targets)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'))

In [55]:
len(char2idx)

20705

In [57]:
char2idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<END>': 2,
 '<UNK>': 3,
 '소문일': 4,
 '예뻐서': 5,
 '이별이지만': 6,
 '화가나네': 7,
 '의자가': 8,
 '너랑': 9,
 '들겠지': 10,
 '우리들': 11,
 '부재가': 12,
 '빠지면': 13,
 '곳이요': 14,
 '자만심': 15,
 '안되겠지만': 16,
 '사셔야겠어요': 17,
 '열어버렸네': 18,
 '열릴': 19,
 '보증금': 20,
 '아이를': 21,
 '무엇으로': 22,
 '쓰레기새끼': 23,
 '귀찮아할까': 24,
 '잡길': 25,
 '유학을': 26,
 '전과': 27,
 '남긴게': 28,
 '덕수궁': 29,
 '생겨요': 30,
 '이끌어주는': 31,
 '쇼핑했더니': 32,
 '고기앞이죠': 33,
 '보내야지': 34,
 '안까먹고': 35,
 '긴가민가한': 36,
 '담배': 37,
 '상대방은': 38,
 '좋아하니까': 39,
 '차렸을': 40,
 '존중하는지': 41,
 '않길': 42,
 '끓여': 43,
 '배낭여행': 44,
 '늙어가': 45,
 '준비해보세요': 46,
 '짧죠': 47,
 '귀여운가봐요': 48,
 '너무견디기힘듭니다': 49,
 '다쳤어': 50,
 '사회를': 51,
 '몰랐구나': 52,
 '별다른': 53,
 '홀로술한잔하며': 54,
 '상하면': 55,
 '추천해드립니다': 56,
 '이닦았는데': 57,
 '낭만이라고는': 58,
 '봐보세요': 59,
 '상메에도': 60,
 '털까': 61,
 '줄여보세요': 62,
 '후폭풍이': 63,
 '가는데': 64,
 '전기': 65,
 '다칠': 66,
 '놓구': 67,
 '명절': 68,
 '감당이': 69,
 '무엇인지': 70,
 '사용해보세요': 71,
 '나와주라': 72,
 '찍었으면': 73,
 '좋아질': 74,
 '딴짓하게돼': 75,
 '난다면': 76,
 '심호흡해보세요': 77,
 '낭만이